<a href="https://colab.research.google.com/github/BluRaf/tree-classify/blob/master/basic-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install tflite-model-maker

In [ ]:
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import image_classifier
from tflite_model_maker import ImageClassifierDataLoader
from tflite_model_maker import model_spec

import matplotlib.pyplot as plt

In [ ]:
import pathlib
data = pathlib.Path('/content/drive/MyDrive/studia/sem5/pk/dataset') 
count = len(list(data.glob('*/*.jpg')))
count

1647

In [ ]:
data = ImageClassifierDataLoader.from_folder("/content/drive/MyDrive/studia/sem5/pk/dataset")
train_data, test_data = data.split(0.7)

INFO:tensorflow:Load image with size: 1647, num_label: 5, labels: akacja, brzoza, dab, lipa, wierzba.


In [ ]:
model = image_classifier.create(train_data)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              3413024   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 3,419,429
Trainable params: 6,405
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5
46/46 [==============================] - 387s 8s/step - loss: 1.2751 - accuracy: 0.5421
Epoch 2/5
46/46 [==============================] - 78s 2s/step - loss: 0.5690 - accuracy: 0.9601
Epoch 3/5
46/46 [==============================] - 76s 2s/step - loss: 0.5254 - accuracy: 0.9700
Epoch 4/5
46/46 [=================

In [ ]:
loss, accuracy = model.evaluate(test_data)

6/6 [==============================] - 45s 6s/step - loss: 0.5020 - accuracy: 0.9818


In [ ]:
model.export(export_dir='.', with_metadata=False)

INFO:tensorflow:Assets written to: /tmp/tmp43orxyjw/assets


INFO:tensorflow:Assets written to: /tmp/tmp43orxyjw/assets


INFO:tensorflow:Saving labels in ./labels.txt.


INFO:tensorflow:Saving labels in ./labels.txt.


In [ ]:
model.evaluate_tflite('model.tflite', test_data)

INFO:tensorflow:Processing example: #0
[[[[0.9882353  1.         1.        ]
   [0.99607843 1.         1.        ]
   [0.99607843 1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[0.9882353  1.         1.        ]
   [0.9882353  1.         1.        ]
   [0.99607843 1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[0.9882353  1.         1.        ]
   [0.9882353  1.         1.        ]
   [0.9882353  1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  ...

  [[0.60672325 0.66610706 0.50644344]
   [0.63929576 0.6986796  0.53901595]
   [0.6162464  0.6756303  0.5159666 ]
   ...
   [0.72692484 0.6939514  0.6514545 ]
   [0.5716701  0.51852876 0.4642674 ]
   [0.57791007 0.51308423 0.4520201 ]]

  [[0.

INFO:tensorflow:Processing example: #0
[[[[0.9882353  1.         1.        ]
   [0.99607843 1.         1.        ]
   [0.99607843 1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[0.9882353  1.         1.        ]
   [0.9882353  1.         1.        ]
   [0.99607843 1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  [[0.9882353  1.         1.        ]
   [0.9882353  1.         1.        ]
   [0.9882353  1.         1.        ]
   ...
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]]

  ...

  [[0.60672325 0.66610706 0.50644344]
   [0.63929576 0.6986796  0.53901595]
   [0.6162464  0.6756303  0.5159666 ]
   ...
   [0.72692484 0.6939514  0.6514545 ]
   [0.5716701  0.51852876 0.4642674 ]
   [0.57791007 0.51308423 0.4520201 ]]

  [[0.

{'accuracy': 0.9818181818181818}